# Full pipeline one step at a time

In [1]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from DLC_for_WBFM.utils.feature_detection.utils_features import *
from DLC_for_WBFM.utils.feature_detection.utils_tracklets import *
from DLC_for_WBFM.utils.feature_detection.utils_detection import *
from DLC_for_WBFM.utils.feature_detection.visualization_tracks import *
from DLC_for_WBFM.utils.video_and_data_conversion.import_video_as_array import *


In [9]:
# Get the 3d bigtiff folder
bigtiff_folder = r'D:\More-stabilized-wbfm'

btf_fname_red = r'test2020-10-22_16-15-20_test4-channel-0-pco_camera1\test2020-10-22_16-15-20_test4-channel-0-pco_camera1bigtiff.btf'
btf_fname_red = os.path.join(bigtiff_folder, btf_fname_red)

# Actually import
import_opt = {'num_slices':33, 'alpha':0.15}

dat0_vid = get_single_volume(btf_fname_red, 15, **import_opt)
dat1_vid = get_single_volume(btf_fname_red, 16, **import_opt)

## First, single-volume segmentation

In [27]:
opt = {'num_slices':33, 'alpha':1.0}
# Build point clouds for each plane
all_keypoints_pcs0 = build_point_clouds_for_volume(dat0_vid, **import_opt)
all_icp0 = build_correspondence_icp(all_keypoints_pcs0)

all_keypoints_pcs1 = build_point_clouds_for_volume(dat0_vid, **import_opt)
all_icp1 = build_correspondence_icp(all_keypoints_pcs1)

[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.
[Open3D WARNING] [KDTreeFlann::SetRawData] Failed due to no data.


In [28]:
all_neurons = [k.points for k in all_keypoints_pcs0]
all_matches = [m.correspondence_set for m in all_icp0]
clust_df0 = build_tracklets_from_matches(all_neurons, all_matches)

all_neurons = [k.points for k in all_keypoints_pcs1]
all_matches = [m.correspondence_set for m in all_icp1]
clust_df1 = build_tracklets_from_matches(all_neurons, all_matches)

In [26]:
i = 1
print(np.asarray(all_neurons[i]))
print(np.asarray(all_neurons[i+1]))

print(np.array(all_matches[i]))

[]
[[  2.         230.60269165 588.4866333 ]]
[[0 1]]


In [29]:
# Visualize
# Build full point cloud

full_pc = o3d.geometry.PointCloud()
for pc in all_keypoints_pcs0:
    full_pc = full_pc + pc
full_pc.paint_uniform_color([0.5,0.5,0.5])

visualize_clusters_from_dataframe(full_pc, clust_df0)

In [30]:
# Visualize
# Build full point cloud

full_pc = o3d.geometry.PointCloud()
for pc in all_keypoints_pcs1:
    full_pc = full_pc + pc
full_pc.paint_uniform_color([0.5,0.5,0.5])

visualize_clusters_from_dataframe(full_pc, clust_df1)

## Next, matching

In [36]:
opt = {'num_slices':33, 'alpha':1.0, 'verbose':1}
neurons0, df0, icp0, pcs0 = detect_neurons_using_ICP(dat0_vid, **opt)
neurons1, df1, icp1, pcs1 = detect_neurons_using_ICP(dat1_vid, **opt)

Building pairwise correspondence...
Building clusters...
Finished ID'ing neurons
Building pairwise correspondence...
Building clusters...
Finished ID'ing neurons


In [37]:
all_features0, all_features1, kp0, kp1 = build_features_on_all_planes(dat0_vid,dat1_vid,
                                                                      verbose=1,
                                                                      matches_to_keep=0.8,
                                                                      num_features_per_plane=10000,
                                                                      detect_keypoints=True,
                                                                      kp0=neurons0,
                                                                      kp1=neurons1)

In [38]:
all_matches, f2n, all_conf = match_centroids_using_tree(np.array(neurons0), 
                               np.array(neurons1), 
                               all_features0, 
                               all_features1,
                               radius=8,
                               max_nn=50,
                               min_features_needed=5,
                                   verbose=1,
                                     to_mirror=False)

Could not match neuron 0
Matched neuron 1 based on 9 features
Could not match neuron 2
Matched neuron 3 based on 20 features
Matched neuron 4 based on 34 features
Could not match neuron 5
Matched neuron 6 based on 6 features
Matched neuron 7 based on 50 features
Matched neuron 8 based on 47 features
Could not match neuron 9
Matched neuron 10 based on 10 features
Matched neuron 11 based on 26 features
Matched neuron 12 based on 32 features
Matched neuron 13 based on 40 features
Matched neuron 14 based on 50 features
Could not match neuron 15
Matched neuron 16 based on 50 features
Matched neuron 17 based on 9 features
Matched neuron 18 based on 6 features
Could not match neuron 19
Could not match neuron 20
Matched neuron 21 based on 50 features
Matched neuron 22 based on 24 features
Matched neuron 23 based on 46 features
Matched neuron 24 based on 7 features
Matched neuron 25 based on 37 features
Matched neuron 26 based on 26 features
Matched neuron 27 based on 35 features
Matched neuron

In [39]:
visualize_tracks(neurons0, neurons1, all_matches, to_plot_failed_lines=False)

# Now, the same thing but with the unified function

In [41]:
from DLC_for_WBFM.utils.feature_detection.feature_pipeline import *


In [43]:
opt = {'num_slices':33, 'alpha':0.15, 'verbose':3}

all_matches, all_conf, all_neurons = track_neurons_full_video(btf_fname_red,
                                                             start_frame=15,
                                                             num_frames=2,
                                                             **opt)

Matching frames 15 and 16 (end at 17)
Building pairwise correspondence...
Building clusters...
Finished ID'ing neurons
Building pairwise correspondence...
Building clusters...
Finished ID'ing neurons
Could not match neuron 0
Matched neuron 1 based on 9 features
Could not match neuron 2
Matched neuron 3 based on 20 features
Matched neuron 4 based on 50 features
Could not match neuron 5
Matched neuron 6 based on 47 features
Matched neuron 7 based on 34 features
Could not match neuron 8
Matched neuron 9 based on 6 features
Matched neuron 10 based on 6 features
Matched neuron 11 based on 32 features
Matched neuron 12 based on 10 features
Matched neuron 13 based on 50 features
Could not match neuron 14
Matched neuron 15 based on 9 features
Matched neuron 16 based on 26 features
Matched neuron 17 based on 50 features
Matched neuron 18 based on 40 features
Matched neuron 19 based on 24 features
Matched neuron 20 based on 7 features
Matched neuron 21 based on 46 features
Matched neuron 22 base

In [44]:
clust_df = build_tracklets_from_matches(all_neurons, all_matches, verbose=1)

0 / 1


In [45]:
i=0

neurons0 = all_neurons[i]
neurons1 = all_neurons[i+1]
matches = all_matches[i]

visualize_tracks(neurons0, neurons1, matches)

In [55]:
# Build full point cloud

pc0 = o3d.geometry.PointCloud()
pc0.points = o3d.cpu.pybind.utility.Vector3dVector(neurons0)
pc0.paint_uniform_color([0.5,0.5,0.5])

pc1 = o3d.geometry.PointCloud()
pc1.points = o3d.cpu.pybind.utility.Vector3dVector(neurons1)
pc1.paint_uniform_color([0.5,0.5,0.5])

full_pc = pc0 + pc1

In [58]:
final_colors = visualize_clusters_from_dataframe(full_pc, clust_df)

In [57]:
full_pc

PointCloud with 282 points.

In [60]:
clust_df

,clust_ind,all_ind_local,all_ind_global,all_xyz,slice_ind,extended_this_slice,not_finished,colors
0,0,"[1, 13]","[1, 155]","[[7.0, 408.03192138671875, 249.4340251575817],...","[0, 1]",True,True,"[0.12156862745098039, 0.4666666666666667, 0.70..."
1,1,"[4, 24]","[4, 166]","[[9.0, 384.00681559244794, 229.6750233968099],...","[0, 1]",True,True,"[0.12156862745098039, 0.4666666666666667, 0.70..."
2,2,"[6, 6]","[6, 148]","[[10.0, 508.24971424449575, 103.97156108509411...","[0, 1]",True,True,"[0.12156862745098039, 0.4666666666666667, 0.70..."
3,3,"[7, 5]","[7, 147]","[[7.5, 513.4677276611328, 98.7389030456543], [...","[0, 1]",True,True,"[0.12156862745098039, 0.4666666666666667, 0.70..."
4,4,"[9, 15]","[9, 157]","[[8.5, 417.65782928466797, 288.3392143249512],...","[0, 1]",True,True,"[0.12156862745098039, 0.4666666666666667, 0.70..."
...,...,...,...,...,...,...,...,...
98,98,"[132, 138]","[132, 280]","[[28.0, 437.0381077357701, 196.5035400390625],...","[0, 1]",True,True,"[0.6196078431372549, 0.8549019607843137, 0.898..."
99,99,"[136, 129]","[136, 271]","[[29.0, 413.2535138811384, 261.9003601074219],...","[0, 1]",True,True,"[0.6196078431372549, 0.8549019607843137, 0.898..."
100,100,"[137, 135]","[137, 277]","[[28.5, 394.19122314453125, 243.80647659301758...","[0, 1]",True,True,"[0.6196078431372549, 0.8549019607843137, 0.898..."
101,101,"[138, 134]","[138, 276]","[[29.5, 400.4838053385417, 254.37733205159506]...","[0, 1]",True,True,"[0.6196078431372549, 0.8549019607843137, 0.898..."
